In [1]:
import cv2
import numpy as np
import os 
os.environ['CUDA_VISIBLE_DEVICES'] = '-1'  # Setting it to -1 disables GPU
from tensorflow.keras.models import load_model
import tensorflow_hub as hub

# Assuming `hub.KerasLayer` is the custom layer
custom_objects = {'KerasLayer': hub.KerasLayer}

face_classifier = cv2.CascadeClassifier(r'C:\Users\msi\Emotion Recognition\haarcascade_frontalface_default.xml')
classifier = load_model(r'C:\Users\msi\Emotion Recognition\model2.h5', custom_objects=custom_objects)

emotion_labels = ['Angry', 'Disgust', 'Fear', 'Happy', 'Neutral', 'Sad', 'Surprise']


# Load train set:

In [2]:
data_dir='..\\Emotion Recognition\\images\\validation'

In [3]:
import pathlib
data_dir = pathlib.Path(data_dir)
data_dir

WindowsPath('../Emotion Recognition/images/validation')

In [4]:
emotion_images_dict = {
    'angry': list(data_dir.glob('angry/*')),
    'disgust': list(data_dir.glob('disgust/*')),
    'fear': list(data_dir.glob('fear/*')),
    'happy': list(data_dir.glob('happy/*')),
    'neutral': list(data_dir.glob('neutral/*')),
    'sad': list(data_dir.glob('sad/*')),
    'surprise': list(data_dir.glob('surprise/*'))
}

In [5]:
emotion_labels_dict = {
    'angry': 0,
    'disgust': 1,
    'fear': 2,
    'happy': 3,
    'neutral': 4,
    'sad' : 5,
    'surprise' :6
}

In [6]:
X, y = [], []

for emotion_name, images in emotion_images_dict.items():
    for image in images:
        img = cv2.imread(str(image))
        resized_img = cv2.resize(img,(224,224))
        X.append(resized_img)
        y.append(emotion_labels_dict[emotion_name])

In [7]:
X = np.array(X)
y = np.array(y)

In [8]:
X = X.astype('float32')

In [9]:
X.shape

(7066, 224, 224, 3)

In [10]:
X_scaled = X / 255.0

# evaluate the train set: 

In [11]:
classifier.evaluate(X_scaled,y)

221/221 [==============================] - 59s 262ms/step - loss: 1.3211 - acc: 0.5116


[1.3210643529891968, 0.5116048455238342]

# fitting the train set to the model to get higher accuracy before testing in real time:

In [12]:
classifier.fit(X_scaled,y , epochs=5)

Epoch 1/5
221/221 [==============================] - 58s 254ms/step - loss: 1.3432 - acc: 0.5091
Epoch 2/5
221/221 [==============================] - 61s 276ms/step - loss: 1.2734 - acc: 0.5286
Epoch 3/5
221/221 [==============================] - 61s 275ms/step - loss: 1.2347 - acc: 0.5456
Epoch 4/5
221/221 [==============================] - 64s 289ms/step - loss: 1.1956 - acc: 0.5568
Epoch 5/5
221/221 [==============================] - 62s 279ms/step - loss: 1.1686 - acc: 0.5701


In [15]:
classifier.save('../Emotion Recognition/model2.h5')

# Real time testing :

In [16]:
classifier = load_model(r'C:\Users\msi\Emotion Recognition\model2.h5', custom_objects=custom_objects)

In [17]:
import cv2
import numpy as np
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input

cap = cv2.VideoCapture(0)

while True:
    _, frame = cap.read()
    labels = []
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = face_classifier.detectMultiScale(gray)

    for (x, y, w, h) in faces:
        cv2.rectangle(frame, (x, y), (x+w, y+h), (0, 255, 255), 2)
        roi_gray = gray[y:y+h, x:x+w]
        roi_color = frame[y:y+h, x:x+w]

        # Resize the face region to match the input size of your model (224, 224, 3)
        roi_resized = cv2.resize(roi_color, (224, 224))
        roi_resized = cv2.cvtColor(roi_resized, cv2.COLOR_BGR2RGB)  # Ensure correct color format
        roi_resized = img_to_array(roi_resized)
        roi_resized = preprocess_input(roi_resized)
        roi_resized = np.expand_dims(roi_resized, axis=0)

        prediction = classifier.predict(roi_resized)[0]
        label = emotion_labels[prediction.argmax()]
        label_position = (x, y)
        cv2.putText(frame, label, label_position, cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)

    cv2.imshow('Emotion Detector', frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


1/1 [==============================] - 0s 34ms/step
